In [1]:
# Import sqlite3 module
import sqlite3

In [161]:
# Chetna-Connect to the playtime database
db=sqlite3.connect("/Users/livestrong19/APAD/APAD_Project1/PickupSport.db")
cursor = db.cursor()

OperationalError: unable to open database file

In [2]:
# Betty-Connect to the playtime database
db=sqlite3.connect("/Users/peihsinho/sqlite/db/PickupSport.db")
cursor = db.cursor()

In [208]:
# Drop Venues table
cursor.execute('''DROP TABLE Venues''')
db.commit()

In [209]:
# Create Venues table
cursor.execute('''
    CREATE TABLE Venues
                        (VenueId INTEGER PRIMARY KEY AUTOINCREMENT, 
                        VenueName TEXT NOT NULL,
                        VenueZipcode INTEGER NOT NULL,
                        VenueAddress TEXT NOT NULL,
                        VenueOpeningTime TIME NOT NULL,
                        VenueClosingTime TIME NOT NULL,
                        VenueTimeSlots INTEGER NOT NULL,
                        VenueSports TEXT,
                        CHECK (LENGTH(VenueZipcode)==5 AND LENGTH(VenueName)<=15 AND LENGTH(VenueAddress)<=30))
                        ''')
db.commit()

In [128]:
# Create Users table

cursor.execute('''
    CREATE TABLE users(uid INTEGER PRIMARY KEY AUTOINCREMENT, uname TEXT,
                       uphone TEXT, uemail TEXT, upassword TEXT, uZipcode TEXT, admin TEXT)
''')
db.commit()

## delete the table
# cursor.execute('''DROP TABLE users''')
# db.commit()

OperationalError: database is locked

In [207]:
# Create Events table
cursor.execute('''
    CREATE TABLE Events
                        (EventId INTEGER PRIMARY KEY AUTOINCREMENT, 
                        EventName TEXT NOT NULL,
                        EventDate DATE NOT NULL,
                        EventSport TEXT NOT NULL,
                        EventTeamSize INTEGER NOT NULL,
                        EventSpotsLeft INTEGER NOT NULL,
                        EventDesc TEXT,
                        VenueId INTEGER,
                        EventHost TEXT NOT NULL,
                        EventStartTime TIME NOT NULL,
                        EventEndTime TIME NOT NULL,
                        FOREIGN KEY (EventHost) references users(uid),
                        FOREIGN KEY (VenueId) REFERENCES Venues(VenueId))
''')
db.commit()

## delete the table
# cursor.execute('''DROP TABLE Events''')
# db.commit()

OperationalError: database is locked

In [18]:
# Create User_Events table
cursor.execute('''
    CREATE TABLE User_Events
                        (EventId Integer,
                         uid Integer,
                         FOREIGN KEY (EventId) REFERENCES Events(EventId), 
                         FOREIGN KEY (uid) REFERENCES users(uid))
''')
db.commit()

## delete the table
# cursor.execute('''DROP TABLE Events''')
# db.commit()

In [17]:
##create admin first
def checkAdmin(uid):
    cursor.execute('''SELECT uid FROM users where admin = "True"''')
    all = cursor.fetchall()
    for user in all:
        if uid in user:
            IsAdmin = True
            return IsAdmin
        else:
            IsAdmin = False         
    return IsAdmin

In [31]:
def addUser(userId, uname, uphone, uemail, uassword, uZipcode, admin):
    
    if checkAdmin(userId) == True:
        value = [(uname, uphone, uemail, upassword, uZipcode, admin)]
        cursor.executemany('''INSERT INTO users(uname, uphone, uemail, upassword, uZipcode ,admin) 
                      VALUES(?, ?, ?, ?, ?, ?)''', value)
        db.commit()
        print("Add the user success!")
        print("User information-"+"uname:"+uname+" uphone: "+uphone+
              " uemail: "+uemail+" uZipcode: "+uZipcode+" admin: "+ admin)
    else:
        print("User is not authorised to add a new user")

In [218]:
def CreateEvent(userId, VenueName, EventName,EventDate, EventSport, EventTeamSize, 
                EventSpotsLeft, EventDesc, EventStartTime,EventEndTime):
    
    cursor.execute('''SELECT uname FROM users WHERE uid = ? ''', (userId,))
    db.commit()
    EventHost = cursor.fetchone()[0]
    print(EventHost) 
    cursor.execute('''SELECT VenueId FROM Venues WHERE VenueName = ? ''', (VenueName,))
    VenueId = cursor.fetchone()[0]
    print(VenueId)
    
    value = [(EventName,EventDate, EventSport, EventTeamSize, 
                EventSpotsLeft, EventDesc, VenueId, EventHost, EventStartTime,EventEndTime)]
    cursor.executemany('''INSERT INTO Events(EventName,EventDate, EventSport, EventTeamSize, 
                EventSpotsLeft, EventDesc, VenueId, EventHost, EventStartTime,EventEndTime) 
                      VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', value)
    db.commit()

CreateEvent(2,'wrtg','arg','argg','arg',5, 5,'argrg','argr', 'srthgt')

vfvsgfb
1


In [37]:
def removeEvent(userId, EventName, EventDate):
    if checkAdmin(userId) == True:
        cursor.execute('''SELECT * FROM Events WHERE EventName = ? and EventDate = ?''', (EventName,EventDate,))
        removedEvent = cursor.fetchone()
        if removedEvent != None:
            cursor.execute('''DELETE FROM Events WHERE EventName = ? and EventDate = ?''', (EventName,EventDate,))
            db.commit()
            print("Remove the event success!")
            print("Removed event information- ", removedEvent)
        else:
            print("No such event!")
    else:
        print("User is not authorised to remove a event")

In [29]:
def addVenue(userId, VenueName, VenueZipcode, VenueAddress, VenueOpeningTime, VenueClosingTime, VenueTimeSlots, VenueSports):
    
    if checkAdmin(userId) == True:
        value = [(VenueName, VenueZipcode, VenueAddress, VenueOpeningTime, VenueClosingTime, VenueTimeSlots, VenueSports)]
        cursor.executemany('''INSERT INTO Venues(VenueName, VenueZipcode, VenueAddress, VenueOpeningTime, VenueClosingTime, VenueTimeSlots, VenueSports) 
                      VALUES(?, ?, ?, ?, ?, ?,?)''', value)
        db.commit()
        print("Add Venue succes!")
    else:
        print("User is not authorised to add a new venue")

In [15]:
#admin create user
uname = input("Enter your name: ")
uphone = input("Enter your phone: ")
uemail = input("Enter your email: ")
upassword = input("Enter your password: ")
uZipcode = input("Enter your uZipcode: ")
admin = input("Enter user admin(True/False):")
addUser(3, uname, uphone, uemail, upassword, uZipcode, admin);

Enter your name: 3567
Enter your phone: sthtb
Enter your email: s5y5
Enter your password: ay
Enter your uZipcode: 45666
Enter user admin(True/False):False
Add user success!
User information-uname:3567/n uphone: sthtb uemail: s5y5 uZipcode: 45666 admin: False


In [30]:
addVenue(3,'arg','78705','dfghjklrtyu','23:50:12','23:50:12',5,'stg')

Add Venue succes!


In [222]:
#add Event
VenueName = input("Enter VenueName: ")
EventName = input("Enter your EventName: ")
EventDate = input("Enter your EventDate: ")
EventSport = input("Enter your EventSport: ")
EventTeamSize = input("Enter your EventTeamSize: ")
EventSpotsLeft = input("Enter your EventSpotsLeft: ")
EventDesc = input("Enter EventDesc: ")
EventStartTime = input("Enter your EventStartTime: ")
EventEndTime = input("Enter EventEndTime: ")

CreateEvent(3,VenueName, EventName,EventDate,EventSport, EventTeamSize, EventSpotsLeft, 
            EventDesc,EventStartTime, EventEndTime)

Enter VenueName: happy
Enter your EventName: erg
Enter your EventDate: erg
Enter your EventSport: reg
Enter your EventTeamSize: 12
Enter your EventSpotsLeft: 5
Enter EventDesc: th
Enter your EventStartTime: wrtg
Enter EventEndTime: 55y
gt
2


In [39]:
removeEvent(4, 'errr', 'gwtg')

User is not authorised to remove a event
